In [1]:
#|default_exp mask_s2

# Masks S2

In [2]:
#|hide
from nbdev.showdoc import *

In [3]:
#|hide
#|export

import geopandas as gpd
import seaborn as sns
from shapely.geometry import Point
import json
import ee

ee.Authenticate()
ee.Initialize(project='ee-reutkeller')

from gee_projects import util_func as utils
from gee_projects import const_vars as const

In [ ]:
#|export

def get_s2_sr_cld_col(fc : ee.featurecollection , # FeatureCollection with geometry datata
                      start_date : str , # Start date for searching images. , 
                      end_date : str , # End date for searching images.
                      ):
    # Import and filter S2 SR.
    s2_sr_col = (ee.ImageCollection(const.s2_img_collection_str)
        .filterBounds(fc)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.lte(const.CLOUDY_PIXEL_PERCENTAGE_STR, const.CLOUD_FILTER)))


    # Import and filter s2cloudless.
    s2_cloudless_col = (ee.ImageCollection(const.COPERNICUS_S2_CLOUD_PROBABILITY_STR)
        .filterBounds(fc)
        .filterDate(start_date, end_date))
    
    # Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
    return ee.ImageCollection(ee.Join.saveFirst(const.s2cloudless_str).apply(**{
        'primary': s2_sr_col,
        'secondary': s2_cloudless_col,
        'condition': ee.Filter.equals(**{
            'leftField': 'system:index',
            'rightField': 'system:index'
        })
    }))


